# 실험 05-01 BJT의 전압-전류 특성

## 공통 코드

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from scipy.signal import savgol_filter # 사비츠키-골레이(Savitzky-Golay) 필터
from numpy import nan, inf

# %matplotlib tk
# %matplotlib inline

def tangent_line(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return lambda t: d*t - d*x + f(x)

def get_simulation_result(file_name, start=0, end=-1):
    with open(file_name, encoding='cp1252') as data_file:
        lines = data_file.read()
        occurrences = lines.count('Step Information:')
        data_file.seek(0)

        line = data_file.readline()
        labels = re.split(', | ,|\t', line)
        labels = [s.strip().upper() for s in labels]

        data = {}

        if (occurrences == 0):
            for label in labels:
                data[label] = []
            
            for line in data_file:
                values = re.split(', | ,|\t', line)
                for i in range(len(values)):
                    value = float(values[i]) * 1000
                    data[labels[i]].append(value)

        else:
            labels_all =[]
            
            lines = data_file.readline() # skip first line starting with 'Step Information:'

            for idx in range(occurrences):
                labels_new = []
                for label in labels:
                    labels_new.append('(%s)@%d' % (label, idx+1))
                labels_all += labels_new

                for label_new in labels_new:
                    data[label_new] = []

                for line in data_file:
                    if (line.startswith('Step Information:')):
                        break
                    values = re.split(', | ,|\t', line)
                    for i in range(len(values)):
                        value = float(values[i]) * 1000
                        data[labels_new[i]].append(value)

            labels = labels_all

        for label in labels:
            data[label] = np.array(data[label][start:end])

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def get_oscilloscpoe_result_tektronix(file_name, start=0, end=-1):
    df = pd.read_csv(file_name, header=None, encoding='cp1252',low_memory=False)

    label_ri = df.loc[df.iloc[:,0] == 'Source', 1].index
    ci = 0
    data = {}
    data['TIME'] = df.iloc[start:end, 3].to_numpy().astype(float)
    data['TIME'] -= data['TIME'][0]
    for idx in range(df.shape[1]//6):
        label = df.iloc[label_ri, ci+1].item()
        data[label] = df.iloc[start:end,ci+4].to_numpy().astype(float)
        ci += 6

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def print_array(label, values):
    print('%s = [' % label, end='')
    for idx, vd in enumerate(values):
        print('{:11.3f}'.format(vd), end='')
        if (idx+1 != len(values)):
            print(', ', end='')
    print(']')

def print_value(label, value):
    print('%s = ' % label, end='')
    print('{:11.3f}'.format(value))

def draw_plot(xs, ys, label, style_idx, marker_num=16, scatter=False, scatter_s=2):
    linestyle  = ['-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-']
    colors     = ['blue',       'green',        'red',          'orange',       'purple',
                  'cyan',       'darkseagreen', 'brown',        'goldenrod',    'darkviolet',
                  'steelblue',  'limegreen',    'tomato',       'tan',          'deeppink',
                  'navy',       'lightgreen',   'indianred',    'khaki',        'rebeccapurple',
                  'slategray',  'forestgreen',  'orangered',    'wheat',        'orchid']
    markers    = ['o',          'v',            '<',            's',            'p',
                  'h',          '*',            'X',            'x',            '^',
                  '>',          'P',            'D',            'H',            'd',
                  '|',          '-',            '4'             '5',            '6',
                  '7',          '8',            '9 ',           '10',           '+']

    if (marker_num > 0):
        markeverys = []
        if xs[0] > xs[-1]:
            first_idx = -1
        else:
            first_idx = 0
        x_step = (xs.max() - xs.min()) / marker_num
        for idx in range(len(markers)):
            x_start = xs.min() + ((x_step / (len(markers) + 1)) * idx)
            marker_list = []
            for j in range(marker_num):
                tmp_array = np.where(xs > (x_start + (x_step * j)))
                if (len(tmp_array[0]) > 0):
                    marker_list.append(tmp_array[0][first_idx])
            markeverys.append(marker_list)

        selected_markevery = markeverys[style_idx]
        selected_marker = markers[style_idx]
    elif (marker_num == 0):
        selected_markevery = None
        selected_marker = None
    else:
        selected_markevery = None
        selected_marker = markers[style_idx]

    if (scatter):
        plt.scatter(xs, ys, color=colors[style_idx], marker=selected_marker, s=scatter_s, label=label)
    else:
        plt.plot(xs, ys, ls=linestyle[style_idx], color=colors[style_idx], marker=selected_marker, markevery=selected_markevery, label=label)
    if (label != None):
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

def set_plot_size(xrate, yrate):
    if (len(set_plot_size.g_plt_figsize) == 0):
        set_plot_size.g_plt_figsize = plt.rcParams["figure.figsize"]
    size = []
    size.append(set_plot_size.g_plt_figsize[0] * xrate)
    size.append(set_plot_size.g_plt_figsize[1] * yrate)
    plt.figure(figsize=size)

set_plot_size.g_plt_figsize = []

*******************************************************************************
## 시뮬레이션 1 결과 분석

## 회로도

* 사용한 컴포넌트
    + voltage (V): VBB, VCE
    + res (R): RB
    + Bipolar NPN transistor (Component -> npn) : Q1
    + ground (G): Ground
    + Net Name (F4): VB, VBB, VCE

![Markdown-mark](../ltspice/simulation_01.png)

### 1. 시뮬레이션 결과 파일을 읽는다.

In [ ]:
data = get_simulation_result('../ltspice/simulation_01.txt')

### 2. 시뮬레이션 결과에서 VCE, IC, IE를 가져온다.

In [ ]:
vcesl = []
icsl = []
iesl = []
labels = [1000., 2000., 3000., 4000., 5000.] # VBB

for idx in range(len(labels)):
    vcesl.append(data['(V(VCE))@%d' % (idx+1)])
    icsl.append(data['(IC(Q1))@%d' % (idx+1)])
    iesl.append(-data['(IE(Q1))@%d' % (idx+1)])

vce_list = [100., 400., 800., 2000., 4000., 6000.]
print_array("                VCE_mV", vce_list)
for idx in range(len(labels)):
    ic_list = []
    for vce in vce_list:
        ic_list.append(np.interp(vce, vcesl[idx], icsl[idx]))
    label = "VBB = {:9.3f}: IC_mA".format(labels[idx])
    print_array(label, ic_list)

### 3. VCE - IC 그래프를 그린다.

In [ ]:
plt.xlabel('VCE_mV')
plt.ylabel('I_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
for idx in range(len(labels)):
    draw_plot(vcesl[idx], icsl[idx], 'IC(VBB=%dmV)' % labels[idx], idx)
plt.show()

### 3. VBB각 5000mV일때 VCE - IC IE 그래프를 그린다.

In [ ]:
plt.xlabel('VCE_mV')
plt.ylabel('I_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
idx = 4
draw_plot(vcesl[idx], icsl[idx], 'IC(VBB=%dmV)' % labels[idx], 0)
draw_plot(vcesl[idx], iesl[idx], 'IE(VBB=%dmV)' % labels[idx], 1)
plt.show()

In [ ]:
plt.xlabel('VCE_mV')
plt.ylabel('I_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
idx = 4
draw_plot(vcesl[idx], icsl[idx], 'IC(VBB=%dmV)' % labels[idx], 0)
draw_plot(vcesl[idx], iesl[idx], 'IE(VBB=%dmV)' % labels[idx], 1)
ax = plt.gca()
ax.set_xlim([5000, 6000])
ax.set_ylim([6.5, 7.5])
plt.show()

*******************************************************************************
## 시뮬레이션 2 결과 분석

## 회로도

![Markdown-mark](../ltspice/simulation_02.png)

### 1. 시뮬레이션 결과 파일을 읽는다.

In [ ]:
data = get_simulation_result('../ltspice/simulation_02.txt')

### 2. 시뮬레이션 결과에서 VBB, VB, IC, IE, IB를 가져온다.

In [ ]:
vbbs = data['V(VBB)']
vbs = data['V(VB)']
ibs = data['IB(Q1)']
ics = data['IC(Q1)']
ies = -data['IE(Q1)']

In [ ]:
vb_list = []
ib_list = []
ic_list = []
ie_list = []

vbb_list = [200., 400., 600., 800., 1000., 2000., 3000., 4000.]
print_array("VBB_mV  ", vbb_list)

for vbb in vbb_list:
    vb_list.append(np.interp(vbb, vbbs, vbs))
    ib_list.append(np.interp(vbb, vbbs, ibs))
    ic_list.append(np.interp(vbb, vbbs, ics))
    ie_list.append(np.interp(vbb, vbbs, ies))
print_array('VBE_mV  ', vb_list)
print_array('IB_mA   ', ib_list)
print_array('IC_mA   ', ic_list)
print_array('IE_mA   ', ie_list)

In [ ]:
beta_dc_list = np.concatenate([np.array([nan, nan, nan,]), np.array(ic_list[3:])/np.array(ib_list[3:])])
print_array('Beta_DC ', beta_dc_list)

alpha_dc_list = np.concatenate([np.array([nan, nan,]), np.array(ic_list[2:])/np.array(ie_list[2:])])
print_array('Alpha_DC', alpha_dc_list)

### 3. VBE - IC 그래프를 그린다.

In [ ]:
plt.xlabel('VBE_mV')
plt.ylabel('I_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(vbs, ics, 'IC_mA', 0)
plt.show()

### 4. VBE - IC IE, VBE - IB 그래프를 그린다.

In [ ]:
plt.xlabel('VBE_mV')
plt.ylabel('I_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(vbs, ics, 'IC_mA', 0)
draw_plot(vbs, ies, 'IE_mA', 1)
plt.show()

In [ ]:
plt.xlabel('VBE_mV')
plt.ylabel('I_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(vbs, ics, 'IC_mA', 0)
draw_plot(vbs, ies, 'IE_mA', 1)
ax = plt.gca()
ax.set_xlim([710, 720])
ax.set_ylim([6, 7])
plt.show()

In [ ]:
plt.xlabel('VBE_mV')
plt.ylabel('I_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(vbs, ibs, 'IB_mA', 2)
plt.show()

*******************************************************************************
## 시뮬레이션 3 결과 분석

## 회로도

* 사용한 컴포넌트
    + voltage (V): VBB, VCE
    + res (R): RB
    + Bipolar PNP transistor (Component -> pnp) : Q1
    + ground (G): Ground
    + Net Name (F4): VB, VBB, VCE

![Markdown-mark](../ltspice/simulation_03.png)

### 1. 시뮬레이션 결과 파일을 읽는다.

In [ ]:
data = get_simulation_result('../ltspice/simulation_03.txt')

### 2. 시뮬레이션 결과에서 VCE, IC, IE를 가져온다.

In [ ]:
vcesl = []
icsl = []
iesl = []
labels = [-1000., -2000., -3000., -4000., -5000.] # VBB

for idx in range(len(labels)):
    vcesl.append(data['(V(VCE))@%d' % (idx+1)])
    icsl.append(data['(IC(Q1))@%d' % (idx+1)])
    iesl.append(-data['(IE(Q1))@%d' % (idx+1)])

vce_list = [-100., -400., -800., -2000., -4000., -5000.]
print_array("                VCE_mV", vce_list)
for idx in range(len(labels)):
    ic_list = []
    for vce in vce_list:
        ic_list.append(np.interp(-vce, -vcesl[idx], icsl[idx])) # x축의 기울기가 음수이기 때문에 -를 붙여서 양수로 바꿈
    label = "VBB = {:9.3f}: IC_mA".format(labels[idx])
    print_array(label, ic_list)

### 3. VCE - IC 그래프를 그린다.

In [ ]:
plt.xlabel('VCE_mV')
plt.ylabel('I_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
for idx in range(len(labels)):
    draw_plot(vcesl[idx], icsl[idx], 'IC(VBB=%dmV)' % labels[idx], idx)
plt.show()

### 3. VBB각 5000mV일때 VCE - IC IE 그래프를 그린다.

In [ ]:
plt.xlabel('VCE_mV')
plt.ylabel('I_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
idx = 4
draw_plot(vcesl[idx], icsl[idx], 'IC(VBB=%dmV)' % labels[idx], 0)
draw_plot(vcesl[idx], iesl[idx], 'IE(VBB=%dmV)' % labels[idx], 1)
plt.show()

In [ ]:
plt.xlabel('VCE_mV')
plt.ylabel('I_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
idx = 4
draw_plot(vcesl[idx], icsl[idx], 'IC(VBB=%dmV)' % labels[idx], 0)
draw_plot(vcesl[idx], iesl[idx], 'IE(VBB=%dmV)' % labels[idx], 1)
ax = plt.gca()
ax.set_xlim([-6000, -5000])
ax.set_ylim([-5., -4.])
plt.show()

*******************************************************************************
## 시뮬레이션 4 결과 분석

## 회로도

![Markdown-mark](../ltspice/simulation_04.png)

### 1. 시뮬레이션 결과 파일을 읽는다.

In [ ]:
data = get_simulation_result('../ltspice/simulation_04.txt')

### 2. 시뮬레이션 결과에서 VBB, VB, IC, IE, IB를 가져온다.

In [ ]:
vbbs = data['V(VBB)']
vbs = data['V(VB)']
ibs = data['IB(Q1)']
ics = data['IC(Q1)']
ies = -data['IE(Q1)']

In [ ]:
vb_list = []
ib_list = []
ic_list = []
ie_list = []

vbb_list = [-200., -400., -600., -800., -1000., -2000., -3000., -4000.]
print_array("VBB_mV ", vbb_list)
for vbb in vbb_list:
    vb_list.append(np.interp(-vbb, -vbbs, vbs)) # x축의 기울기가 음수이기 때문에 -를 붙여서 양수로 바꿈
    ib_list.append(np.interp(-vbb, -vbbs, ibs))
    ic_list.append(np.interp(-vbb, -vbbs, ics))
    ie_list.append(np.interp(-vbb, -vbbs, ies))
print_array('VBE_mV ', vb_list)
print_array('IB_mA  ', ib_list)
print_array('IC_mA  ', ic_list)
print_array('IE_mA  ', ie_list)

In [ ]:
beta_dc_list = np.concatenate([np.array([nan, nan, nan,]), np.array(ic_list[3:])/np.array(ib_list[3:])])
print_array('Beta_DC ', beta_dc_list)

alpha_dc_list = np.concatenate([np.array([nan, nan,]), np.array(ic_list[2:])/np.array(ie_list[2:])])
print_array('Alpha_DC', alpha_dc_list)

### 3. VBE - IC 그래프를 그린다.

In [ ]:
plt.xlabel('VBE_mV')
plt.ylabel('I_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(vbs, ics, 'IC_mA', 0)
plt.show()

### 4. VBE - IC IE, VBE - IB 그래프를 그린다.

In [ ]:
plt.xlabel('VBE_mV')
plt.ylabel('I_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(vbs, ics, 'IC_mA', 0)
draw_plot(vbs, ies, 'IE_mA', 1)
plt.show()

In [ ]:
plt.xlabel('VBE_mV')
plt.ylabel('I_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(vbs, ics, 'IC_mA', 0)
draw_plot(vbs, ies, 'IE_mA', 1)
ax = plt.gca()
ax.set_xlim([-743, -733])
ax.set_ylim([-4.5, -3.5])
plt.show()

In [ ]:
plt.xlabel('VBE_mV')
plt.ylabel('I_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(vbs, ibs, 'IB_mA', 2)
plt.show()